# Chicago Airbnb Data Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import random
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from PIL import Image
import requests
from io import BytesIO
from collections import Counter

#### Business questions

- Q1: How do listing information differ among different neiborhoods?
- Q2: Is there a general upward trend of both new Airbnb listings and total Airbnb visitors to Chicago?
- Q3: What are the busiest times of a year to visit Chicago? By how much do prices spike?
- Q4: What are the factors that explain the listing price the most?

## 1. Data gathering

### 1.1. Load the data

Airbnb data: Chicago, Illinois, United States <br>
Data source: http://insideairbnb.com/get-the-data.html

- listings.csv.gz:	Detailed Listings data for Chicago
- calendar.csv.gz:	Detailed Calendar Data for listings in Chicago
- reviews.csv.gz:	Detailed Review Data for listings in Chicago
- listings.csv:	Summary information and metrics for listings in Chicago (good for visualisations).
- reviews.csv:	Summary Review data and Listing ID (to facilitate time based analytics and visualisations linked to a listing).
- neighbourhoods.csv:	Neighbourhood list for geo filter. Sourced from city or open source GIS files.

In [2]:
# Listings
listing = pd.read_csv("listings.csv.gz")
print("Listings table:")
print(f"rows, cols: {listing.shape}")
listing.head()

Listings table:
rows, cols: (7666, 96)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2384,https://www.airbnb.com/rooms/2384,20181011131034,2018-10-11,Hyde Park-Walk to UChicago or Theological Semi...,"As the sole guest in my quiet, vintage (1924) ...","The spacious bedroom has a queen size bed, che...","As the sole guest in my quiet, vintage (1924) ...",none,My building is located one block from beautifu...,...,t,City registration pending,"{""Illinois State"","" Cook County"","" IL"","" CHICA...",f,f,strict_14_with_grace_period,f,f,1,2.91
1,4505,https://www.airbnb.com/rooms/4505,20181011131034,2018-10-11,1 Great Apartment. 352 Great Reviews. 1 bad one.,Across the street from CTA train. Runs every 6...,"We travel a lot, we know what people need. We...",Across the street from CTA train. Runs every 6...,none,NaN,...,t,City registration pending,"{""Illinois State"","" Cook County"","" IL"","" CHICA...",t,f,moderate,f,f,1,3.11
2,6715,https://www.airbnb.com/rooms/6715,20181011131034,2018-10-11,Lincoln Park Oasis - Unit 2 ONLY,Unit 1 & Unit 2 are rented separately. They ca...,License #: (Phone number hidden by Airbnb) Be...,Unit 1 & Unit 2 are rented separately. They ca...,none,Things To Do & Close to: - An awesome Children...,...,t,2114275,"{""Illinois State"","" Cook County"","" IL"","" CHICA...",f,f,strict_14_with_grace_period,f,f,2,0.82
3,9811,https://www.airbnb.com/rooms/9811,20181011131034,2018-10-11,Barbara's Hideaway - Old Town,One-bedroom hideaway tucked into Old Town step...,"This lovely one bedroom ""hideaway"" is located ...",One-bedroom hideaway tucked into Old Town step...,none,Chicago’s Old Town neighborhood is squeezed be...,...,t,2079260,"{""Illinois State"","" Cook County"","" IL"","" CHICA...",t,f,strict_14_with_grace_period,f,f,8,0.55
4,10610,https://www.airbnb.com/rooms/10610,20181011131034,2018-10-11,3 Comforts of Cooperative Living,The condo is the 2nd floor in a lovely 1912 3-...,Newly furnished with queen bed and the comfort...,The condo is the 2nd floor in a lovely 1912 3-...,none,It's a 10 minute walk from the lakefront bike ...,...,t,City registration pending,"{""Illinois State"","" Cook County"","" IL"","" CHICA...",t,f,moderate,f,f,5,0.64


In [3]:
# Listings summary
listing_summary = pd.read_csv("listings.csv")
print("Listings summay table:")
print(f"rows, cols: {listing_summary.shape}")
listing_summary.head()

Listings summay table:
rows, cols: (7666, 16)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,Hyde Park-Walk to UChicago or Theological Semi...,2613,Rebecca,NaN,Hyde Park,41.788865,-87.586709,Private room,50,2,133,2018-10-01,2.91,1,241
1,4505,1 Great Apartment. 352 Great Reviews. 1 bad one.,5775,Craig & Kathleen,NaN,South Lawndale,41.854953,-87.696962,Entire home/apt,120,2,363,2018-09-24,3.11,1,188
2,6715,Lincoln Park Oasis - Unit 2 ONLY,15365,Reem,NaN,Lincoln Park,41.929262,-87.660091,Entire home/apt,255,4,93,2018-08-12,0.82,2,352
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,NaN,Lincoln Park,41.917689,-87.637879,Entire home/apt,150,3,30,2018-08-13,0.55,8,349
4,10610,3 Comforts of Cooperative Living,2140,Lois And Ed,NaN,Hyde Park,41.797085,-87.591949,Private room,35,2,31,2018-07-29,0.64,5,144


In [4]:
# Calendar
calendar = pd.read_csv("calendar.csv.gz")
print("Calendar table:")
print(f"rows, cols: {calendar.shape}")
calendar.head()

Calendar table:
rows, cols: (2798090, 4)


,listing_id,date,available,price
0,2384,2019-07-27,f,NaN
1,2384,2019-07-26,f,NaN
2,2384,2019-07-25,f,NaN
3,2384,2019-07-24,f,NaN
4,2384,2019-07-23,f,NaN


In [5]:
# Reviews
review = pd.read_csv("reviews.csv.gz")
print("Reviews table:")
print(f"rows, cols: {review.shape}")
review.head()

Reviews table:
rows, cols: (259883, 6)


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2384,25218143,2015-01-09,14385014,Ivan,it's a wonderful trip experience. I didn't exc...
1,2384,28475392,2015-03-24,16241178,Namhaitou,This is my first trip using Airbnb. I was a li...
2,2384,30273263,2015-04-19,26101401,Patrick,The reservation was canceled 80 days before ar...
3,2384,30974202,2015-04-30,26247321,Cristina,Sólo puedo decir cosas buenas de Rebecca. La h...
4,2384,31363208,2015-05-04,31293837,SuJung,Rebecca was an absolutely wonderful host.\r\n\...


In [6]:
# Reviews summary
review_summary = pd.read_csv("reviews.csv")
print("Review summay table:")
print(f"rows, cols: {review_summary.shape}")
review_summary.head()

Review summay table:
rows, cols: (259883, 2)


,listing_id,date
0,2384,2015-01-09
1,2384,2015-03-24
2,2384,2015-04-19
3,2384,2015-04-30
4,2384,2015-05-04


In [7]:
# Neighborhood
neighborhood = pd.read_csv("neighbourhoods.csv")
print("Neighborhood table:")
print(f"rows, cols: {neighborhood.shape}")
neighborhood.head()

Neighborhood table:
rows, cols: (77, 2)


,neighbourhood_group,neighbourhood
0,NaN,Albany Park
1,NaN,Archer Heights
2,NaN,Armour Square
3,NaN,Ashburn
4,NaN,Auburn Gresham


### 1.2. Relational join

Join reviews to listings by combining reviews for each listing.

In [8]:
# Prepare review data for join
review = review.fillna(' ')
review = review.groupby(by=['listing_id'])[['comments']].sum()

# Join dataframe with reviews
listing12 = listing.merge(review, how='left', left_on='id', right_index=True)

### 1.3. Feature design and cleaning

In [9]:
# Drop irrelavent information for the scope of this study
# Note some of these features have only 1 unique value
# Note some are duplicated features
listing13 = listing12.drop(columns=['listing_url', 'last_scraped', 'street', 'host_url', 'host_name', 'host_location', 
                                  'neighbourhood_cleansed', 'city', 'state', 'market', 'access', 'host_thumbnail_url',
                                  'smart_location', 'country_code', 'country', 'experiences_offered',
                                  'calendar_last_scraped', 'calendar_updated', 'summary', 'host_neighbourhood',
                                  'host_verifications', 'has_availability', 'first_review', 'last_review',
                                  'license', 'jurisdiction_names', 'is_business_travel_ready', 
                                  'thumbnail_url', 'medium_url', 'xl_picture_url', 'space', 'scrape_id',
                                  'host_acceptance_rate', 'neighbourhood_group_cleansed', 'host_total_listings_count'
                                 ])

In [10]:
# Numeric features
print("Numeric features:")
listing13.dtypes.loc[listing13.dtypes!='object'].index

Numeric features:


Index(['id', 'host_id', 'host_listings_count', 'latitude', 'longitude',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'square_feet',
       'guests_included', 'minimum_nights', 'maximum_nights',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value',
       'calculated_host_listings_count', 'reviews_per_month'],
      dtype='object')

#### Strategies for numeric features

- 'id', 'host_id': drop before machine learning
- All other features: keep

In [11]:
# Non-numeric features
print("Non-numeric features")
listing13.dtypes.loc[listing13.dtypes=='object'].index

Non-numeric features


Index(['name', 'description', 'neighborhood_overview', 'notes', 'transit',
       'interaction', 'house_rules', 'picture_url', 'host_since', 'host_about',
       'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_picture_url', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood', 'zipcode', 'is_location_exact', 'property_type',
       'room_type', 'bed_type', 'amenities', 'price', 'weekly_price',
       'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people',
       'requires_license', 'instant_bookable', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'comments'],
      dtype='object')

#### Strategies for converting non-numeric features

- Before exploratory data analysis

    - 'name', 'notes', 'transit', 'interaction', 'house_rules': word count
    - 'description', 'neighborhood_overview', 'host_about', 'comments': sentiment and word count
    - 'picture_url', 'host_picture_url': darkness and colorfulness
    - 'host_since': days till today, drop the original column before machine learning
    - 'host_response_rate', 'host_response_time', 'price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people': convert to values
    - 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'is_location_exact', 'requires_license', 'instant_bookable', 'require_guest_profile_picture', 'require_guest_phone_verification': dummy
    - 'amenities': break the set then conver to dummies

- After exploratory data analysis

    - 'neighbourhood': keep for exploratory data analysis, drop for machine learning or one hot encoding
    - 'zipcode': ordinal numbers or one hot encoding
    - 'property_type', 'room_type', 'bed_type', 'cancellation_policy': one hot encoding

#### Word count

In [12]:
def word_count(series):
    '''
    Count the number of words in a string
    Input - string, text
    Output - int, number of words
    '''
    corpus = series
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    count = np.sum(X.toarray(), axis=1)
    return count

In [13]:
word_count_cols = ['name', 'notes', 'transit', 'interaction', 'house_rules', 'description', 
                   'neighborhood_overview', 'host_about', 'comments']
word_count_names = [col + '_word_count' for col in word_count_cols]

# Replace NaN by '' in order to do word count
listing13[word_count_cols] = listing13[word_count_cols].fillna('')

# Do word count for each relavent feature
for i, col in enumerate(word_count_cols):
    listing13[word_count_names[i]] = word_count(listing13[col])

# Drop original word count columns
listing13 = listing13.drop(columns=['name', 'notes', 'transit', 'interaction', 'house_rules'])

#### Sentiment analysis

In [14]:
analyzer = SentimentIntensityAnalyzer()
def sentiment(target_string):
    '''
    Analyze the sentiment of a string
    Input - string, text
    Output - float, sentiment compound score
    '''
    try:
        results = analyzer.polarity_scores(target_string)
        compound = results["compound"]
    except:
        compound = target_string
    return compound

In [15]:
sentiment_cols = ['description', 'neighborhood_overview', 'host_about', 'comments']
sentiment_names = [col + '_sentiment' for col in sentiment_cols]

# Do sentiment analysis for each relavent feature
for i, col in enumerate(sentiment_cols):
    listing13[sentiment_names[i]] = listing13[col].apply(sentiment)

# Drop original sentiment columns
listing13 = listing13.drop(columns=sentiment_cols)

In [16]:
# Save checkpoint
listing13.to_csv("temp/listing13_sentiment.csv", index=False)

#### Image analysis

In [17]:
# Check for NaN in image URLs
listing13[['picture_url', 'host_picture_url']].isnull().sum() # No NaN

picture_url         0
host_picture_url    0
dtype: int64

In [18]:
# Fetch images from urls
picture_imgs = []
for url in listing13['picture_url']:
    try:
        response = requests.get(url)
        img = np.array(Image.open(BytesIO(response.content)))
        picture_imgs.append(img)
    except:
        picture_imgs.append(np.nan)
    
host_picture_imgs = []
for url in listing13['picture_url']:
    try:
        response = requests.get(url)
        img = np.array(Image.open(BytesIO(response.content)))
        host_picture_imgs.append(img)
    except:
        host_picture_imgs.append(np.nan)

In [19]:
def img_darkness(list_of_arr):
    '''
    Calculated the darkness and colorfulness of image arrays
    Input - list of image arrays
    Output - tuple of a list of image darkness and a list of image colorfulness. 
             Higher darkness value means lighter color, lower value means darker color.
             Higher colorfulness value means more color variations.
    '''
    dark_list = []
    colorful_list = []
    
    for img in list_of_arr:
        try:
            darkness = np.mean(img)
            colorfulness = np.mean(np.std(img, axis=(0,1)))
            dark_list.append(darkness)
            colorful_list.append(colorfulness)
        except:
            dark_list.append(img)
            colorful_list.append(img)
            
    return dark_list, colorful_list

In [20]:
# Calculate darkness and colorfulness from images
pic_dark, pic_colorful = img_darkness(picture_imgs)
host_pic_dark, host_pic_colorful = img_darkness(host_picture_imgs)

In [21]:
# Add to the dataframe
listing13['pic_dark'], listing13['pic_colorful'], listing13['host_pic_dark'], listing13['host_pic_colorful'] \
= pic_dark, pic_colorful, host_pic_dark, host_pic_colorful

# Drop original urls
listing13 = listing13.drop(columns=['picture_url', 'host_picture_url'])
listing13[['pic_dark', 'pic_colorful', 'host_pic_dark', 'host_pic_colorful']].head()

,pic_dark,pic_colorful,host_pic_dark,host_pic_colorful
0,126.704685,59.293558,126.704685,59.293558
1,154.640696,79.402266,154.640696,79.402266
2,113.128704,54.522272,113.128704,54.522272
3,139.048470,49.004923,139.048470,49.004923
4,112.844087,45.904076,112.844087,45.904076


In [22]:
# Save checkpoint
listing13.to_csv("temp/listing13_image.csv", index=False)

#### Datetime

In [23]:
listing13 = pd.read_csv("temp/listing13_image.csv")
listing13['host_since'].head()

0    2008-08-29
1    2008-12-29
2    2009-04-30
3    2009-08-21
4    2008-08-16
Name: host_since, dtype: object

In [24]:
# Convert datetime to the number of days till today
listing13['host_days'] = (pd.to_datetime('today') - pd.to_datetime(listing13['host_since'], 
                                                                   format='%Y-%m-%d')).dt.days
# listing13 = listing13.drop(columns=['host_since'])
listing13['host_since'] = pd.to_datetime(listing13['host_since'], format='%Y-%m-%d')

#### To numeric

In [25]:
listing13[['host_response_rate', 'host_response_time', 'price', 'weekly_price', 
           'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people']].head()

,host_response_rate,host_response_time,price,weekly_price,monthly_price,security_deposit,cleaning_fee,extra_people
0,NaN,NaN,$50.00,$350.00,NaN,$0.00,$20.00,$0.00
1,100%,within an hour,$120.00,$925.00,NaN,$0.00,$30.00,$25.00
2,NaN,NaN,$255.00,"$1,850.00","$5,300.00",$500.00,$90.00,$10.00
3,100%,within an hour,$150.00,"$1,050.00","$3,400.00",NaN,$95.00,$25.00
4,100%,within an hour,$35.00,$235.00,$805.00,$200.00,$50.00,$10.00


In [26]:
listing13['host_response_time'].value_counts()

within an hour        4117
within a few hours     572
within a day           331
a few days or more      36
Name: host_response_time, dtype: int64

In [27]:
strip_cols = [
    'host_response_rate', 
    'price', 
    'weekly_price', 
           'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people']

for col in strip_cols:
    listing13[col] = listing13[col].str.strip('%$').str.replace(',', '').astype('float')

listing13[['host_response_rate', 'host_response_time', 'price', 'weekly_price', 
           'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people']].head()

,host_response_rate,host_response_time,price,weekly_price,monthly_price,security_deposit,cleaning_fee,extra_people
0,NaN,NaN,50.0,350.0,NaN,0.0,20.0,0.0
1,100.0,within an hour,120.0,925.0,NaN,0.0,30.0,25.0
2,NaN,NaN,255.0,1850.0,5300.0,500.0,90.0,10.0
3,100.0,within an hour,150.0,1050.0,3400.0,NaN,95.0,25.0
4,100.0,within an hour,35.0,235.0,805.0,200.0,50.0,10.0


In [28]:
# Replace description with approximate numbers
listing13['host_response_time'] = listing13['host_response_time'].replace({
    'within an hour': 1,
    'within a few hours': 6,
    'within a day': 24,
    'a few days or more': 6
})

#### To dummy

In [29]:
dummy_cols = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'is_location_exact', 
              'requires_license', 'instant_bookable', 'require_guest_profile_picture', 
              'require_guest_phone_verification']
listing13[dummy_cols].head()

,host_is_superhost,host_has_profile_pic,host_identity_verified,is_location_exact,requires_license,instant_bookable,require_guest_profile_picture,require_guest_phone_verification
0,t,t,t,t,t,f,f,f
1,t,t,t,t,t,t,f,f
2,f,t,f,t,t,f,f,f
3,f,t,t,t,t,t,f,f
4,f,t,t,t,t,t,f,f


In [30]:
listing13[dummy_cols] = listing13[dummy_cols].replace({
    't': 1,
    'f': 0
})

#### Amenities

In [31]:
def break_amen(value):
    amen_list = value.strip('{}').split(',')
    amen_list = [item.strip('""') for item in amen_list]
    return amen_list

# Number of amenity items
listing13['amenities_count'] = listing13['amenities'].apply(break_amen).apply(len)

In [32]:
# Get all amenity items
amen_list = []
for value in listing13['amenities']:
    amen_list.extend(break_amen(value))

# Sanity check
Counter(amen_list).most_common()

[('Essentials', 7480),
 ('Wifi', 7461),
 ('Heating', 7447),
 ('Smoke detector', 7269),
 ('Kitchen', 7154),
 ('Air conditioning', 6928),
 ('Hangers', 6744),
 ('Shampoo', 6658),
 ('Carbon monoxide detector', 6456),
 ('Hair dryer', 6332),
 ('TV', 6268),
 ('Iron', 6210),
 ('Laptop friendly workspace', 6005),
 ('Washer', 5934),
 ('Dryer', 5917),
 ('Hot water', 5299),
 ('Fire extinguisher', 4869),
 ('Self check-in', 3879),
 ('First aid kit', 3866),
 ('Free street parking', 3734),
 ('Refrigerator', 3706),
 ('Dishes and silverware', 3530),
 ('Microwave', 3481),
 ('Stove', 3369),
 ('Oven', 3338),
 ('Bed linens', 3296),
 ('Cooking basics', 3282),
 ('Coffee maker', 3223),
 ('Lock on bedroom door', 3213),
 ('Family/kid friendly', 3162),
 ('Long term stays allowed', 2875),
 ('Extra pillows and blankets', 2568),
 ('Dishwasher', 2438),
 ('Cable TV', 2376),
 ('Private entrance', 2263),
 ('Internet', 2216),
 ('Luggage dropoff allowed', 2179),
 ('Free parking on premises', 2099),
 ('Lockbox', 1897),
 ('

In [33]:
# Unique list of amenity items
amen_unique_list = list(set(amen_list) - {'', 'translation missing: en.hosting_amenity_49', 
                       'translation missing: en.hosting_amenity_50'})

# Break list into columns of boolean
new_cols_bool = np.array([listing13['amenities'].apply(break_amen).map(lambda amens: amen in amens) 
                for amen in amen_unique_list])

# Add new amenity columns to the dataframe
new_col_names = ['amenity_' + value for value in amen_unique_list]
new_cols = pd.DataFrame(new_cols_bool.T, columns=new_col_names).astype(int)
listing13[new_col_names] = new_cols

# Drop original amenity column
listing13 = listing13.drop(columns=['amenities'])

#### Save as a checkpoint for exploratory data analysis

In [34]:
listing13.to_csv("temp/listing13.csv", index=False)
listing13.head()

,id,host_id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood,...,amenity_Bathtub,amenity_Smoking allowed,amenity_Exercise equipment,amenity_Wide entryway,amenity_Rain shower,amenity_Outdoor seating,amenity_Beach essentials,amenity_Disabled parking spot,amenity_Essentials,amenity_Wine cooler
0,2384,2613,2008-08-29,NaN,NaN,1,1,1,1,Hyde Park,...,0,0,0,0,0,0,0,0,1,0
1,4505,5775,2008-12-29,1.0,100.0,1,1,1,1,Little Village,...,1,0,0,0,0,0,0,0,1,0
2,6715,15365,2009-04-30,NaN,NaN,0,2,1,0,Lincoln Park,...,0,0,0,0,0,0,0,0,1,0
3,9811,33004,2009-08-21,1.0,100.0,0,8,1,1,Old Town,...,0,0,0,0,0,0,0,0,1,0
4,10610,2140,2008-08-16,1.0,100.0,0,5,1,1,Hyde Park,...,0,0,0,0,0,0,0,0,1,0


To be continued in `explore_part2.ipynb`